In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!git clone --recursive https://github.com/microsoft/LightGBM 
%cd LightGBM
!mkdir build
%cd build
!cmake -DUSE_GPU=1 ..
!make -j$(nproc)

In [0]:
%cd /content/LightGBM/python-package
!python3 setup.py install --gpu

In [0]:
!pip install category_encoders

In [0]:
import pandas as pd
import numpy as np
import category_encoders as ce

# Model import
#from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import KBinsDiscretizer
#from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.preprocessing import KBinsDiscretizer
from math import sqrt
import lightgbm as lgb

In [0]:
cd /content/drive/My\ Drive

/content/drive/My Drive


In [0]:
# Define paths
training_file_path = "tcd-ml-1920-group-income-train.csv"
validation_file_path = "tcd-ml-1920-group-income-test.csv"
# submission_file_path = "tcd ml 2019-20 income prediction submission file.csv"

# Read training data, identify missing data
missing_values_list = ["unknown", "Unknown", "UNKNOWN", "nA", "na", "n/a", "-", "--", "#NUM!"]
training_data = pd.read_csv(training_file_path, na_values = missing_values_list, dtype={'Housing Situation': str})
validation_data = pd.read_csv(validation_file_path, na_values = missing_values_list, dtype={'Housing Situation': str})

In [0]:
training_data['Primary Income'] = training_data['Total Yearly Income [EUR]'] - training_data['Yearly Income in addition to Salary (e.g. Rental Income)']

renameColumnsWithSquareBracket = {'Body Height [cm]':'Body Height','Work Experience in Current Job [years]':'Work Experience'}
training_data.rename(columns=renameColumnsWithSquareBracket, inplace=True)
validation_data.rename(columns=renameColumnsWithSquareBracket, inplace=True)


In [0]:
fill_col_dict = {'Year of Record': 1999,
 'Gender':'female',
 'Age': 15,
 'Profession': 'principal administrative associate',
 'University Degree': 0,
 'Hair Color': 'Black'
 }
for col in fill_col_dict.keys():
    training_data[col] = training_data[col].fillna(fill_col_dict[col])
    validation_data[col] = validation_data[col].fillna(fill_col_dict[col])


In [0]:
training_data = training_data.ffill(axis = 0)
validation_data = validation_data.ffill(axis = 0)

del_col = set(['Yearly Income in addition to Salary (e.g. Rental Income)','Total Yearly Income [EUR]','Primary Income'])
features_col =  list(set(training_data) - del_col)
training_features = training_data[features_col] #.iloc[:, :-3]
training_target = training_data['Primary Income'] #training_data.iloc[:, -1]

print(training_features.dtypes)

Age                                         int64
Hair Color                                 object
Gender                                     object
Housing Situation                          object
Body Height                                 int64
Crime Level in the City of Employement      int64
University Degree                          object
Profession                                 object
Year of Record                            float64
Size of City                                int64
Instance                                    int64
Satisfation with employer                  object
Wears Glasses                               int64
Work Experience                           float64
Country                                    object
dtype: object


In [30]:
training_data.corr(method='pearson')

,Instance,Year of Record,Crime Level in the City of Employement,Work Experience,Age,Size of City,Wears Glasses,Body Height,Yearly Income in addition to Salary (e.g. Rental Income),Total Yearly Income [EUR],Primary Income
Instance,1.000000,0.998324,-0.000912,-0.000662,-0.000886,0.000091,0.001268,0.000552,0.000645,0.630978,0.645867
Year of Record,0.998324,1.000000,-0.003940,-0.000753,-0.000947,0.000122,0.001220,0.000485,0.000555,0.634595,0.649590
Crime Level in the City of Employement,-0.000912,-0.003940,1.000000,0.196145,0.201478,-0.000845,0.002288,0.000105,-0.001283,-0.046253,-0.047074
Work Experience,-0.000662,-0.000753,0.196145,1.000000,0.968895,0.000218,0.001639,0.054738,0.000238,0.105108,0.107560
Age,-0.000886,-0.000947,0.201478,0.968895,1.000000,0.000396,0.001071,0.000777,0.000647,0.100672,0.102928
Size of City,0.000091,0.000122,-0.000845,0.000218,0.000396,1.000000,-0.001075,-0.000388,-0.002033,0.021637,0.022597
Wears Glasses,0.001268,0.001220,0.002288,0.001639,0.001071,-0.001075,1.000000,0.001380,-0.005266,-0.000428,0.000713
Body Height,0.000552,0.000485,0.000105,0.054738,0.000777,-0.000388,0.001380,1.000000,0.003245,0.020800,0.020585
Yearly Income in addition to Salary (e.g. Rental Income),0.000645,0.000555,-0.001283,0.000238,0.000647,-0.002033,-0.005266,0.003245,1.000000,0.214460,0.000865
Total Yearly Income [EUR],0.630978,0.634595,-0.046253,0.105108,0.100672,0.021637,-0.000428,0.020800,0.214460,1.000000,0.976918


In [0]:
target_encoder = ce.TargetEncoder(cols=['Housing Situation', 'Gender', 'Country','Profession','University Degree',
                                        'Wears Glasses', 'Satisfation with employer', 'Hair Color'], handle_unknown='ignore', min_samples_leaf = 100,smoothing = 10.0)


training_features = target_encoder.fit_transform(training_features, training_target).replace([np.inf, -np.inf], np.nan).dropna(axis = 'rows').astype(int)

train_X, val_X, train_y, val_y = train_test_split(training_features, training_target)

val_y_scaled = np.log(val_y)
train_y_scaled = np.log(train_y)


In [23]:
#------------------------Build LightGBM Model-----------------------
train_data=lgb.Dataset(train_X, categorical_feature=['Housing Situation', 'Gender','University Degree',
                                        'Wears Glasses', 'Satisfation with employer', 'Hair Color'], label=train_y_scaled)
valid_data=lgb.Dataset(val_X, categorical_feature=['Housing Situation', 'Gender','University Degree',
                                        'Wears Glasses', 'Satisfation with employer', 'Hair Color'], label=val_y_scaled)

#Select Hyper-Parameters
params = {'metric' : 'mae',
          'boosting_type' : 'gbdt',
          'learning_rate': 0.001,
          'max_depth' : -1,
          'device_type':'gpu',
          #'max_bin':'63',
          'verbose' : -1,
          'reg_alpha': 2,
          #'min_child_samples': 50, 
          #'min_child_weight': 0.1,
          'reg_lambda': 5,
          #'subsample': 0.855,
          'bagging_seed' : 11,
          'ignore_column' : ['Instance', 'Age', 'Wears Glasses','Crime Level in the City of Employement']
}

#Train model on selected parameters and number of iterations
lgbm = lgb.train(params,
                 train_data,
                 25000,
                 valid_sets=valid_data,
                 early_stopping_rounds= 5,
                 verbose_eval= 1000
                 )

/content/LightGBM/python-package/lightgbm/basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 5 rounds
[1000]	valid_0's l1: 0.612688
[2000]	valid_0's l1: 0.329008
[3000]	valid_0's l1: 0.235088
[4000]	valid_0's l1: 0.196199
[5000]	valid_0's l1: 0.176966
[6000]	valid_0's l1: 0.166845
[7000]	valid_0's l1: 0.160814
[8000]	valid_0's l1: 0.156521
[9000]	valid_0's l1: 0.153473
[10000]	valid_0's l1: 0.151258
[11000]	valid_0's l1: 0.149499
[12000]	valid_0's l1: 0.147972
[13000]	valid_0's l1: 0.146879
[14000]	valid_0's l1: 0.145886
[15000]	valid_0's l1: 0.145011
[16000]	valid_0's l1: 0.144191
[17000]	valid_0's l1: 0.143503
[18000]	valid_0's l1: 0.142846
[19000]	valid_0's l1: 0.142275
[20000]	valid_0's l1: 0.141746
[21000]	valid_0's l1: 0.141283
[22000]	valid_0's l1: 0.140872
[23000]	valid_0's l1: 0.1405
[24000]	valid_0's l1: 0.140154
[25000]	valid_0's l1: 0.139846
Did not meet early stopping. Best iteration is:
[25000]	valid_0's l1: 0.139846


In [24]:
target_pred = np.exp(lgbm.predict(val_X))
print(mean_absolute_error(val_y, target_pred, sample_weight=None, multioutput='uniform_average'));

9107.301871941521


In [0]:
extra_income = validation_data['Yearly Income in addition to Salary (e.g. Rental Income)']
validation_data = target_encoder.transform(validation_data.iloc[:, :-2])
validation_data['Total Yearly Income [EUR]'] = np.exp(lgbm.predict(validation_data)) + extra_income
validation_data.to_csv("result1.csv", index=False, columns=["Instance", "Total Yearly Income [EUR]"])